In [1]:
from dotenv import load_dotenv
from openai import OpenAI

In [3]:
_ = load_dotenv(override=True)

In [31]:
client = OpenAI()

In [41]:
import mercury as mr

In [42]:
app = mr.App(title="Text2Results Virtual Assistant")

mercury.App

In [43]:
q_input = mr.Text(label="Question for the dataset", value="What is the average amount of each customer's orders on their first order dates?", sanitize=False)
mr.Note(text="Examples:<br>Find the dates each customer ordered the most amount.<br>Find the dates each customer reached $10 LTV, or null if not reached.")

mercury.Text

In [44]:
button = mr.Button(label="Get Results", style="primary")
button_clicked = button.clicked

mercury.Button

In [37]:
prompt = f"""
You are a helpful assistant that converts natural language questions into BigQuery SQL queries.

Database schema:
Table: jaffle_shop_analytics.dim_customers(customer_id, first_name, last_name, first_order_date, most_recent_order_date, number_of_orders, lifetime_value)
Table: jaffle_shop_analytics.fct_orders(order_id, customer_id, order_date, amount)

SQL Guidelines:
Avoid using subqueries. Use CTEs with good naming conventions. Use CASE WHEN statements in aggregate functions if needed and avoid joining a table more than once.

Question: {q_input.value}

SQL:
"""


In [38]:
if button_clicked:
    completion = client.responses.create(
        model="gpt-4",
        input=[{"role":"user", "content":prompt}],
        temperature=0
    )

    sql_query = completion.output_text
    print(sql_query)

SELECT f.order_id, f.customer_id, f.order_date, f.amount
FROM dbt_ksiu.fct_orders AS f
JOIN dbt_ksiu.dim_customers AS d ON f.customer_id = d.customer_id
WHERE d.lifetime_value > 2;


In [47]:
import sys
import os

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

In [45]:
from bigquery_connection import run_bigquery_query
import pandas as pd

In [40]:
if button_clicked:
    results = run_bigquery_query(sql_query)
    df = pd.DataFrame(results)
    mr.Table(df, width="200px")

[{'order_id': 65,
  'customer_id': 26,
  'order_date': datetime.date(2018, 3, 8),
  'amount': 0.0},
 {'order_id': 83,
  'customer_id': 54,
  'order_date': datetime.date(2018, 3, 24),
  'amount': 1.0},
 {'order_id': 3,
  'customer_id': 94,
  'order_date': datetime.date(2018, 1, 4),
  'amount': 1.0},
 {'order_id': 90,
  'customer_id': 66,
  'order_date': datetime.date(2018, 3, 30),
  'amount': 2.0},
 {'order_id': 17,
  'customer_id': 71,
  'order_date': datetime.date(2018, 1, 18),
  'amount': 2.0},
 {'order_id': 76,
  'customer_id': 25,
  'order_date': datetime.date(2018, 3, 20),
  'amount': 2.0},
 {'order_id': 14,
  'customer_id': 40,
  'order_date': datetime.date(2018, 1, 17),
  'amount': 3.0},
 {'order_id': 32,
  'customer_id': 28,
  'order_date': datetime.date(2018, 2, 4),
  'amount': 3.0},
 {'order_id': 53,
  'customer_id': 34,
  'order_date': datetime.date(2018, 2, 26),
  'amount': 3.0},
 {'order_id': 80,
  'customer_id': 11,
  'order_date': datetime.date(2018, 3, 23),
  'amount': 